<a href="https://colab.research.google.com/github/theocharistr/Natural_Language_Processing/blob/main/Natural_Language_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets



In [ ]:
import datasets
import transformers
import accelerate
from transformers import AutoTokenizer, AutoModelForTokenClassification,TrainingArguments

In [ ]:
# Load the MultiNERD dataset

from datasets import load_dataset

dataset = load_dataset("Babelscape/multinerd")

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
# Filter out non-English examples

english_dataset = dataset.filter(lambda x: x["lang"] == "en")

In [ ]:
english_dataset['train'][0]

{'tokens': ['The', 'type', 'locality', 'is', 'Kīlauea', '.'],
 'ner_tags': [0, 0, 0, 0, 5, 0],
 'lang': 'en'}

In [ ]:
english_dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'lang'],
        num_rows: 262560
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'lang'],
        num_rows: 32820
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'lang'],
        num_rows: 32908
    })
})

In [ ]:
# Tokenize and encode the text
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


In [ ]:
# Define the LM model
model_name = 'bert-base-cased'
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=5)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
!pip install accelerate -U
!pip install transformers[torch]
import accelerate
import transformers

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./outputs",
    num_train_epochs=3,
    per_device_train_batch_size=16,
)

ImportError: ignored

In [ ]:
# Create the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=preprocessed_dataset['train'],
    eval_dataset=preprocessed_dataset['test'],
    tokenizer=tokenizer,
)


NameError: ignored

In [ ]:
# Train the model
trainer.train()

In [ ]:
# Evaluate the model
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")